# How to use a preprocessor in a retrieval chain

In [1]:
from textprepper.modifiers import GoogleTrans

from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import chromadb

## 1. Setup Preprocessor for Language Translation (Example Google Translator)

In [2]:
translator = GoogleTrans(source_lng="auto", target_lng="german") # Translate to german
translator("This is a simple test!")

'Dies ist ein einfacher Test!'

In [3]:
# Using the Translator in a simple chain
prompt = ChatPromptTemplate.from_template("""Text: {input}""")

simple_chain = RunnablePassthrough() | translator | prompt

print(simple_chain.invoke("This is a test."))

messages=[HumanMessage(content='Text: Das ist ein Test.')]


## 2. Using the translator in a Retrieval Chain

- I am using a database which I already setup for another project about two pdfs from the german goverment.
- The focus is here only to show the possible usage in a lcel chain with a retriever.

In [4]:
SIMPLE_RAG = """Answer the question based only on the following context:

{context}

Question: {question}
"""
SIMPLE_RAG_PROMPT = ChatPromptTemplate.from_template(SIMPLE_RAG)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Setup ChromaDB
CHROMA_CLIENT = chromadb.HttpClient(host="localhost", port=8000) # My chromadb is runnning in a container.
database = Chroma(
    client=CHROMA_CLIENT,
    collection_name="openai_embedded",
    embedding_function=embedding_model,
)

## Let's start with a simple query to the database

#### Ask in german about unemployment

As a reminder from the docstrings:
```python
"""
Returns:
    List[Tuple[Document, float]]: List of documents most similar to
    the query text and cosine distance in float for each.
    Lower score represents more similarity.
"""
```

In [5]:
from textprepper.removers import RemoveNewLines
def pretty_print_documents_with_scores(results):
    print("Document Score | Content | page_number")
    for document, score in results:
        shorter_content = RemoveNewLines(count=1)(document.page_content[:80])
        page = document.metadata["page"]
        print(f"{score:.3f} | {shorter_content} | {page}")

In [6]:
ger_results = database.similarity_search_with_score("Ich werde bald arbeitlos und benötige Hilfe. Was muss ich als nächstes tun?", k=10)

In [7]:
pretty_print_documents_with_scores(ger_results)

Document Score | Content | page_number
0.804 | » Leistungen und Angebote der Agentur für Arbeit » Arbeitsuchend und arbeitslos | 14
0.812 | wieder arbeitsuchend zu melden (vgl. » Abschnitt 1 ). Über Fristen und das Verf | 18
0.828 | 2.  Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?2.1 Arbeitslos meldenL | 14
0.838 | einer selbständigen Tätig  keit bzw. bevor  die Kosten entstanden sind, bean  t | 90
0.839 | beschrie  ben – bei einer Agentur für  Arbeit arbeitsuchend melden.  Nähere  I | 13
0.848 | land arbeiten, erhalten Sie bei Arbeits  losigkeit grund ­sätzlich Leistungen v | 15
0.861 | keine Arbeit aufnehmen können. Sprechen Sie mit Ihrer Integrationsfachkraft, we | 20
0.870 | HINWEISKönnen Sie bereits bei Ihrer Arbeitslosmeldung wegen Krankheit keine Be | 28
0.874 | bereit und in der Lage sein, eine Beschäftigung von mindestens 15 Stunden wöche | 59
0.888 | fordert sind, Ihre Beschäftigungslosigkeit zu beenden. Ihre Arbeitsvermittlerin | 20


#### Ask in english the same question

In [8]:
en_results = database.similarity_search_with_score("I will be unemployed soon and need help. What should I do next?", k=10)

In [9]:
pretty_print_documents_with_scores(en_results)

Document Score | Content | page_number
1.176 | land arbeiten, erhalten Sie bei Arbeits  losigkeit grund ­sätzlich Leistungen v | 15
1.186 | Weitere HilfenTeilarbeitslosengeldWenn Sie teilarbeitslos sind, besteht die Mö | 89
1.196 | 2.  Was müssen Sie tun, wenn Arbeitslosigkeit eintritt?2.1 Arbeitslos meldenL | 14
1.196 | wieder arbeitsuchend zu melden (vgl. » Abschnitt 1 ). Über Fristen und das Verf | 18
1.200 | » Leistungen und Angebote der Agentur für Arbeit » Arbeitsuchend und arbeitslos | 14
1.202 | einer selbständigen Tätig  keit bzw. bevor  die Kosten entstanden sind, bean  t | 90
1.222 | haben, setzen Sie sich bitte unverzüglich mit dem für Sie zuständigen Jobcenter | 49
1.228 | die Jobcenter .  HINWEISSollten Sie zeitgleich Arbeitslosengeld von Ihrer Agen | 11
1.232 | HINWEISBeachten Sie hierzu das » Merkblatt Bürgergeld  – Grundsicherung für Ar | 91
1.247 | Wenn Sie während einer Sperrzeit Ihren Lebensunter ­halt aus Ihren vorhandenen  | 58


**You can see, that the scores are different and obviously the german question is more similar to the documents. But you can also recognize, that you will get different documents. Some documents are the same but may have a different "rank" in the order. So depending on the use case, embedding model and/or LLM, it might be beneficial to translate the user's query into the language of the database first to receive better/different results.**

### Let's setup a retrieval chain (without a LLM)

- the documents are in german, so that's why I translate the query into the same language
- For this example I excluded the llm (not in focus)

#### Let's show first how you usually setup a retrieval chain without a translator (comparison)

In [10]:
usual_retrieval_chain = {
    "context":  database.as_retriever() | format_docs,
    "question": RunnablePassthrough(),
} | SIMPLE_RAG_PROMPT

results = usual_retrieval_chain.invoke("I will be unemployed soon and need help. What should I do next?")
print(results.messages[0].content)

Answer the question based only on the following context:

land arbeiten, erhalten Sie bei Arbeits  losigkeit grund ­
sätzlich Leistungen von Ihrem Wohnortstaat. Zur 
 Sicherung möglicher Ansprüche melden Sie sich bitte 
zeitnah beim Träger der Arbeits  losenversicherung in 
Ihrem Wohnortstaat.
2.2 Antrag stellen 
Sie können den Antrag online unter  
» www.arbeitsagentur.de /eservices » Leistungen 
und Angebote der Agentur für Arbeit 
»  Arbeitslosengeld beantragen“  stellen. Es reicht 
aus, wenn Sie den Antrag ca. 2  Wochen vor Eintritt der

Weitere Hilfen
Teilarbeitslosengeld
Wenn Sie teilarbeitslos sind, besteht die Möglichkeit 
Teilarbeitslosengeld zu beziehen. Sie sind teilarbeits ­
los, wenn Sie eine versicherungspflichtige Beschäfti ­
gung, die Sie neben mindestens einer weiteren versicherungspflichtigen Beschäftigung ausgeübt haben, 
verloren haben und eine versicherungspflichtige 
 Beschäftigung suchen. Das » Merkblatt  1a für Teilarbeitslose finden Sie unter: » www.arbeitsagen

##### Let's change the LCEL Chain and integrate the translator

In [11]:
translator_retrieval_chain = {
    "context": translator | database.as_retriever() | format_docs,
    "question": RunnablePassthrough(),
} | SIMPLE_RAG_PROMPT

results = translator_retrieval_chain.invoke("I will be unemployed soon and need help. What should I do next?")
print(results.messages[0].content)

Answer the question based only on the following context:

» Leistungen und Angebote der Agentur für Arbeit 
» Arbeitsuchend und arbeitslos melden.  Oder Sie 
 suchen Ihre Agentur für Arbeit auf, um sich persönlich 
arbeitslos zu melden.
Es ist wichtig, dass Sie sich spätestens am 1. Tag der 
Arbeitslosigkeit arbeitslos melden; dies kann auch 
inner  halb von 3   Monaten vor dem  Beginn der 
 Arbeitslosigkeit  geschehen. Beachten Sie jedoch bitte 
unbedingt die  Hinweise zur gesonderten Arbeitsuchendmeldung in  »  Abschnitt  1.
 HINWEIS

wieder arbeitsuchend zu melden (vgl. » Abschnitt 1 ). 
Über Fristen und das Verfahren für die Arbeitsuchend ­
meldung informieren Sie sich bitte bei Ihrer Agentur für 
Arbeit. Um Nachteile zu vermeiden, müssen Sie sich 
auch erneut – spätestens am ersten Tag der Arbeitslosigkeit – online im Fachportal der Bundesagentur für 
Arbeit oder persönlich bei Ihrer Agentur für Arbeit 
arbeits  los melden (vgl. » Abschnitt 2.1 ).
Üben Sie unentgeltlich eine ehren

##### If you want to return translated question too:

In [12]:
SIMPLE_RAG = """Answer the question based only on the following context:

Original Question: {question}
Translation: {translation}

{context}
"""
RAG_PROMPT = ChatPromptTemplate.from_template(SIMPLE_RAG)

In [13]:
translator_retrieval_chain = {
    "context": translator | database.as_retriever() | format_docs,
    "question": RunnablePassthrough(),
    "translation": translator # Only to show the output
} | RAG_PROMPT

In [14]:
results = translator_retrieval_chain.invoke("I will be unemployed soon and need help. What should I do next?")
print(results.messages[0].content)

Answer the question based only on the following context:

Original Question: I will be unemployed soon and need help. What should I do next?
Translation: Ich werde bald arbeitslos sein und brauche Hilfe. Was soll ich als nächstes tun?

wieder arbeitsuchend zu melden (vgl. » Abschnitt 1 ). 
Über Fristen und das Verfahren für die Arbeitsuchend ­
meldung informieren Sie sich bitte bei Ihrer Agentur für 
Arbeit. Um Nachteile zu vermeiden, müssen Sie sich 
auch erneut – spätestens am ersten Tag der Arbeitslosigkeit – online im Fachportal der Bundesagentur für 
Arbeit oder persönlich bei Ihrer Agentur für Arbeit 
arbeits  los melden (vgl. » Abschnitt 2.1 ).
Üben Sie unentgeltlich eine ehrenamtliche Tätigkeit

» Leistungen und Angebote der Agentur für Arbeit 
» Arbeitsuchend und arbeitslos melden.  Oder Sie 
 suchen Ihre Agentur für Arbeit auf, um sich persönlich 
arbeitslos zu melden.
Es ist wichtig, dass Sie sich spätestens am 1. Tag der 
Arbeitslosigkeit arbeitslos melden; dies kann auch 


## If you want to translate the retrieved document for your final LLM prompt, you can use the DocumentPreprocessorPipe:

In [15]:
from textprepper import DocumentPreprocessorPipe

In [16]:
en_trans = GoogleTrans(target_lng="english")
trans = DocumentPreprocessorPipe([en_trans])

# Use the .from_documents method of the DocumentPreprocessorPipe instance
lcel = RunnablePassthrough() | translator | database.as_retriever() | trans.from_documents

In [17]:
lcel.invoke("Arbeitslosengeld")

[Document(page_content='money, transitional allowance, pension due to full working life\nment or incapacity to work, the old age pension\nthe statutory pension insurance, the Knapp\nsocial compensation benefit or similar public benefits\npublic law), your entitlement to unemployment is suspended\nsengeld in whole or in part. The corresponding\nLegal regulations are intended to prevent “double payment”\nimpede.\nIf you receive parental allowance, this corresponds to the receipt of\nUnemployment benefits are not available. However, you must', metadata={'page': 59, 'rm_header': True, 'section_number': '6.', 'section_title': 'Sperrzeit', 'source': '../pdf_files/merkblatt-fuer-arbeitslose_ba036520.pdf', 'topic': 'Arbeitslosengeld'}),
 Document(page_content='on unemployment benefits even if you are in the', metadata={'page': 29, 'rm_header': True, 'section_number': '2.5', 'section_title': 'Verfügbar sein', 'source': '../pdf_files/merkblatt-fuer-arbeitslose_ba036520.pdf', 'topic': 'Arbeitslos